In [14]:
import numpy as np
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [15]:
f = open("spambase.columns", "r")
headers = [line.split(":")[0] for line in f]+["class"]

df = pd.read_csv("spambase.data", header=None) 
df.columns = headers
df["class"] = df["class"].replace(0,-1)

df

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,class
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,-1
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,-1
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,-1
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,-1


In [16]:
X = np.array(df.drop('class', axis=1))
y = df['class']
X.shape, y.shape

((4601, 57), (4601,))

In [17]:
seed = 42

In [21]:
clf = make_pipeline(StandardScaler(), LinearSVC(random_state=seed, max_iter=1e4, tol=1e-4, penalty='l1', dual=False, C=1.353))
clf.fit(X,y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc',
                 LinearSVC(C=1.353, dual=False, max_iter=10000.0, penalty='l1',
                           random_state=42))])

In [22]:
clf.named_steps['linearsvc'].coef_,  clf.named_steps['linearsvc'].intercept_

(array([[-3.47867310e-02, -3.59285983e-02,  3.12133807e-02,
          8.30386858e-01,  1.36886970e-01,  8.08108122e-02,
          2.54107091e-01,  8.03788359e-02,  6.19587661e-02,
          2.78436944e-02, -1.87164312e-02, -4.44835777e-02,
         -1.22799352e-02,  1.85914205e-02,  1.12329528e-01,
          2.76299073e-01,  1.25017289e-01,  3.40532717e-02,
          5.30694470e-02,  1.07124585e-01,  1.15652421e-01,
          7.89001102e-02,  2.66700401e-01,  6.87120536e-02,
         -7.62778617e-01, -3.41110550e-01, -7.05603972e+00,
          8.11779749e-02, -4.07297585e-01, -5.87079473e-02,
         -1.39788314e-02,  2.21041993e-01, -1.32895561e-01,
          0.00000000e+00, -3.46308593e-01,  1.16469941e-01,
         -5.91428123e-03, -4.18879956e-02, -1.06383983e-01,
         -3.05416308e-02, -2.40255516e+00, -5.38025652e-01,
         -5.78828335e-02, -2.52314096e-01, -2.35458964e-01,
         -3.88050757e-01, -4.96264376e-02, -2.92521630e-01,
         -1.01798614e-01, -1.79530868e-0

In [50]:
non_zero_coef = (np.abs(clf.named_steps['linearsvc'].coef_) > 2e-1).reshape(-1)

In [52]:
np.array(headers[:-1])[non_zero_coef]

array(['word_freq_3d', 'word_freq_remove', 'word_freq_free',
       'word_freq_000', 'word_freq_hp', 'word_freq_hpl',
       'word_freq_george', 'word_freq_lab', 'word_freq_857',
       'word_freq_85', 'word_freq_cs', 'word_freq_meeting',
       'word_freq_project', 'word_freq_re', 'word_freq_edu',
       'word_freq_conference', 'char_freq_$', 'char_freq_#',
       'capital_run_length_longest'], dtype='<U26')